Kaggleのようなデータサイエンスのコンペで結果を出すための基本的な流れをTitanicのコンペを例に紹介していきます。


コンペの基本的な流れは以下のステップです。

・問題の定義の確認する

・trainデータとtestデータの取得する

・データを整理し、準備し、きれいにする(データクレンジング)

・分析、パターンを識別しデータを調査する

・問題をモデル化し、予測する

・結果を提出する

このフローは、必ずこれに従わなくてはならないというわけではなく場合によっては前後しても問題ありません。
それでは順に解説していきます。


In [ ]:
import pandas as pd
import numpy as np
sub1=pd.read_csv('../input/titanic-leaked/titanic.csv')

In [ ]:
sub1.to_csv('submission1.csv', index=False)

In [ ]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

## 0. データの取得
まず、トレーニングデータとテストデータを取得します。効率良く前処理をするためにこれらのデータセットを組み合わせます。


In [ ]:
train_df = pd.read_csv('../input/titanic/train.csv')
test_df = pd.read_csv('../input/titanic/test.csv')
combine = [train_df, test_df]

## 1. 特徴量チェックポイント7つ

### 1-1. データにはどのような特徴量があるか？###

データの特徴量を確認します。基本的にこれらはKaggleデータページに記載されています。


In [ ]:
print(train_df.columns.values)

### 1-2. どの特徴量がカテゴリーデータか？###
カテゴリーデータの特徴量は「名義尺度」「順序尺度」「間隔尺度」「比例尺度」の内、どれに基づいているか確認します。

・カテゴリー特徴量: Survived, Sex, Cabin, Pclass

### 1-3. どの特徴量が数値データか？###
数値特徴量の中で、値は離散的なものなのか、連続的なものなのか、それとも時系列に基づいたものなのか確認します。

・連続的特徴量: Age, Fare

・離散的特徴量: SibSp,Parch


In [ ]:
# preview the data
train_df.head()

### 1-4. カテゴリーと数値が混在している特徴量は？###
カテゴリーと数値が混在している場合は、何かしらの処置が必要です。

・Ticketは数字と英数字のデータ型が混在

・Cabinは英数字

### 1-5. エラーやミスなどが含まれている可能性のある特徴量は？###
大規模なデータセットの場合、全てを確認することが難しいですが小規模なデータセットからいくつかのサンプルを確認することで、どの特徴量が修正する必要があるかわかることがあります。
Nameの特徴量にはタイトル，カッコ，代替名や短い名前に使われる引用符など，いくつかの表現方法があるので，エラーなどが含まれている可能性があるので注意が必要です。


In [ ]:
train_df.tail()

### 1-6. どの特徴量に空欄、NULL、のような欠損値が含まれているか？### 
欠損値がある場合は何かしらの処置が必要です。
Cabin > Age > Embarked これらの特徴量には、trainデータの場合、この順番で欠損値がいくつか含まれています。
テストデータの場合、Cabin > Ageで欠損が多いので補完する必要があります。

### 1-7. 各種特徴量のデータ型は(整数、小数、文字列)のどれに該当するか？### 
整数(int)か小数(float)の特徴量は7つ、
文字列(object)の特徴量は6つあります。


In [ ]:
train_df.info()
print('_'*40)
test_df.info()

## 2. サンプル間の特徴量の数値分布はどのようになっているか？##

・Survivedは0または1の値を持つカテゴリー特徴量

・乗客の約38％が生存していた

・ほとんどの乗客（75％以上）は親子連れではなかった

・乗客の30％近くが兄弟や配偶者を連れていた

・運賃には大きなばらつきがあり、512ドルという高額な料金を支払っている乗客はほとんどいない（1％未満）

・65歳から80歳までの高齢者の乗客はほとんどいない（1％未満）


In [ ]:
train_df.describe()

### 2-1. カテゴリ特徴量の分布は？##

・Nameはデータ全体で一意である。 (count=unique=891)

・Sexは65%が男性。

・Cabinの値は、サンプル間で複数の重複がある。あるいは、複数の乗客がCabinを共有している場合もある。

・Embarkedの値は3つでほとんどの乗客はS港を使用していた。 (top=S)

・Ticketは、重複率が高い(22%) (unique=681)


In [ ]:
train_df.describe(include=['O'])

### 2-2. 特徴量をピボットして分析する##

仮説を確認するために、互いの特徴量を分析することで特徴量の相関を素早く分析することができます。この段階でできるのは欠損値を持たない特徴に対してのみです。また、カテゴリ型（Sex）、順序型（Pclass）、離散型（SibSp, Parch）の特徴量に対してのみ有効です。

・Pclass： Pclass=1とSurvived の間には、有意な相関が見られる(>0.5)。この特徴量をモデルに使用することにする。

・Sex： Sex=女性の生存率が74%と非常に高いということがわかった。

・SibSp と Parch の相関はゼロです。これらの特徴量から新たに特徴量を作成するのも有効かも？。


In [ ]:
train_df[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train_df[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train_df[["SibSp", "Survived"]].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train_df[["Parch", "Survived"]].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)

## 3. データを可視化して分析する##
データを可視化して、いくつかの仮説の確認します。


### 3-1. 数値的特徴(Age)との相関関係##
まず、数値的特徴とSurvivedとの相関関係を理解することから始めましょう。

ヒストグラムは、年齢のような連続した数値変数を分析するのに便利で、範囲が有用なパターンを特定するのに便利です。
ヒストグラムの可視化におけるx軸は、サンプルまたは乗客の数を表しています。

考察結果

・乳児（年齢 <=4）の生存率が高かった。

・最も年配の乗客（年齢 = 80）は生存していた。

・多くの15-25歳の乗客は生存していなかった。

・乗客のほとんどは15-35歳の年齢層である。

意思決定

・Ageはモデル学習の際に考慮すべき特徴量である。

・欠損値を補完し年齢特徴量を完成させる必要がある。

・年齢グループをバンド化したほうがよいかも？


In [ ]:
g = sns.FacetGrid(train_df, col='Survived')
g.map(plt.hist, 'Age', bins=20)

### 3-2. 数値特徴と順序特徴(Pclass)の相関関係###
複数の特徴量を組み合わせ、1つのプロットを使用して相関関係を識別することができます。これは数値を持つ数値特徴とカテゴリ特徴で行うことができます。

考察結果

・Pclass=3の乗客は最も多いがほとんどは生き残っていなかった。

・Pclass=2とPclass=3の幼児の乗客はほとんどが生存していた。

・Pclass=1の乗客のほとんどが生存していた。

・Pclassは乗客の年齢分布によって異なる。

意思決定

・Pclassはモデル学習の際に考慮すべき特徴量である。


In [ ]:
grid = sns.FacetGrid(train_df, col='Survived', row='Pclass', size=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend();

### 3-3. カテゴリー的特量(Sex, Embarked, Pclass)の相関関係 ###
これで、我々は解決の目標とカテゴリの特徴を相関させることができるようになりました。

考察結果

・女性の乗客は男性よりも生存率は高かった。

・例外として、Embarked=Cでは男性の方が生存率が高かった。これは、PclassとEmbarked、PclassとSurvivedの相関関係である可能性があり必ずしもEmbarkedとSurvivedの直接的な相関関係ではない。

・CポートとQポートでは、Pclass=2と比較してPclass=3の方が男性の生存率が高かった。 

・乗船した港は、Pclass=3と男性の間で生存率に差がある。

意志決定

・Sexはモデル学習に必要な特徴量であるから追加する。

・欠損を補完したEmbarkedを特徴量に追加する．


In [ ]:
grid = sns.FacetGrid(train_df, row='Embarked', size=2.2, aspect=1.6)
grid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', palette='deep')
grid.add_legend()

### 3-4. カテゴリー特徴(Embarked, Sex, Fare)と数値特徴の相関 ##
また、（数値以外の値を持つ）カテゴリー特徴量と数値的特徴量を相関させたい場合もあると思います。今回はEmbarked (カテゴリー非数値)、Sex (カテゴリー非数値)、Fare (連続数値)、Survived (カテゴリー数値)との相関を検討してみようと思います。

考察結果

・Fareが高い方が生存率が高かった。

・Embarkedは生存率と相関がある。

意志決定

・Fareをバンド化することを検討。


In [ ]:
grid = sns.FacetGrid(train_df, row='Embarked', col='Survived', size=2.2, aspect=1.6)
grid.map(sns.barplot, 'Sex', 'Fare', alpha=.5, ci=None)
grid.add_legend()

In [ ]:
print("Before", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape)

train_df = train_df.drop(['Ticket', 'Cabin'], axis=1)
test_df = test_df.drop(['Ticket', 'Cabin'], axis=1)
combine = [train_df, test_df]

"After", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape

## 4. 特徴量作成 ##

### 4-1. 既存の機能から抽出した新しい特徴量の作成###
NameとPassengerIDを削除する前に、Nameからタイトルを抽出できるかどうかを分析し、タイトルと生存率の相関関係を調べてみます。

以下のコードで、正規表現を用いてタイトルを抽出します。(\w+\.)は、Nameのドット文字で終わる最初の単語にマッチし、expand=Falseは、DataFrameを返します。

考察結果

Title、Age、およびSurvivedをプロットすると、以下のことが確認できます。

・ほとんどのタイトルは、年齢グループを正確にバンドすることができる。たとえば、Master タイトルの平均年齢は 5 年。

・タイトル間の生存率は年齢で若干の違いがある。

・特定のタイトル（Mme, Lady, Sir）はほとんどが生存していたり、（Don, Rev, Jonkheer）そうでなかったりします。

意志決定

・新しいNameタイトル特徴量を作成する。

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(train_df['Title'], train_df['Sex'])

数の少ないタイトルは一般的な名前に置き換えたり、レアなグループとして分類します。

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
train_df[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

カテゴリのタイトルを順序に変換する。


In [ ]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

train_df.head()

Nameの特徴量はもう必要ないので削除します。また、PassengerId特徴量も必要ないので削除しましょう。


In [ ]:
train_df = train_df.drop(['Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
combine = [train_df, test_df]
train_df.shape, test_df.shape

### 4-2. カテゴリー特徴量の変換###
これで、文字列を含む特徴量を数値に変換できました。このような変換はほとんどのモデルアルゴリズムで必要で、これを行うことで特徴量の完成させます。

まず、Sexを（女性=1、男性=0）に変換してみます。


In [ ]:
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

train_df.head()

### 4-3. 数値連続特徴量の欠損補完###
ここで、欠落値を持つ特徴量の補完をします。まず、年齢特徴量について。

数値連続特徴量の欠損を補完させるには、3つの方法が考えらます。

1. 単純な方法は、平均と標準偏差の間に乱数を生成させ補完する。

2. 欠損値を推測するより正確な方法は、他の相関のある特徴を使用することです。今回はAge、Gender、Pclassの間の相関関係に着目してす。Pclass と Gender を組み合わせと Age の中央値を使用して Age の値を推測します。つまり、Pclass=1 と Gender=0、Pclass=1 と Gender=1 の年齢の中央値などで補完するということです。

3. 方法1と2を組み合わせます。中央値に基づいて年齢を推測するのではなく、PclassとGenderの組み合わせのセットに基づいて、平均と標準偏差の間の乱数を生成させるやり方です。

方法1と3は、モデルにランダム・ノイズを導入することで実装できます。
実行から得られる結果は異なるので今回は2を選択します。


In [ ]:
grid = sns.FacetGrid(train_df, row='Pclass', col='Sex', size=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend()

まず、Pclass x Gender の組み合わせから推測される年齢の値を格納する空の配列を用意します。


In [ ]:
guess_ages = np.zeros((2,3))
guess_ages

ここで、Sex (0 または 1) と Pclass (1, 2, 3) を反復処理して、6つの組み合わせでAgeの推測値を計算します。


In [ ]:
for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Pclass'] == j+1)]['Age'].dropna()

            age_guess = guess_df.median()

            # Convert random age float to nearest .5 age
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]

    dataset['Age'] = dataset['Age'].astype(int)

train_df.head()

年齢のバンドを作成して、Survivedとの相関を調べてみます。


In [ ]:
train_df['AgeBand'] = pd.cut(train_df['Age'], 5)
train_df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

これらのバンドに基づいて、年齢を通常値に変換します。


In [ ]:
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']
train_df.head()

In [ ]:
train_df = train_df.drop(['AgeBand'], axis=1)
combine = [train_df, test_df]
train_df.head()

### 4-4. 既存の特徴量を組み合わせて新しい特徴量を作成する###
ParchとSibSpを組み合わせたFamilySizeという新しい特徴量を作成します。これにより、データセットから Parch と SibSp を削除できるようになります。

In [ ]:
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

train_df[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False)

IsAlone という別の特徴量を作成します。


In [ ]:
for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

train_df[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

Parch、SibSp、FamilySize の特徴量を削除して、IsAlone を使用する。


In [ ]:
train_df = train_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
test_df = test_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
combine = [train_df, test_df]

train_df.head()

また、PclassとAgeを組み合わせた特徴量を作成することもできます。


In [ ]:
for dataset in combine:
    dataset['Age*Class'] = dataset.Age * dataset.Pclass

train_df.loc[:, ['Age*Class', 'Age', 'Pclass']].head(10)

### 4-5. カテゴリー特徴の完成###
特徴量Embarkedは，乗船した港に基づいて S, Q, C の値を取る。今回のtrainデータの欠損値は最頻値で補完します。


In [ ]:
freq_port = train_df.Embarked.dropna().mode()[0]
freq_port

In [ ]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
    
train_df[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

### 4-6. カテゴリー特徴量を数値に変換する###
Embarkedを数値に変換します。


In [ ]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

train_df.head()

### 4-7. 数値特徴量を完成させ変換する###
これで、テストデータ内の単一の欠落値のFare特徴量を、この特徴量に対して最頻値で補完するためにモードを使用して完成させた。

1つの値を置き換えるだけなので、新しい特徴を作成したり、欠落した特徴を推測するための相関関係のための更なる分析する必要はありません。目標はモデルアルゴリズムが問題なく動くよう欠損値をなくすことです。

また、運賃は通貨を表しているので、小数点以下2桁に丸めた方が良いかもしれません。


In [ ]:
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)
test_df.head()

In [ ]:
train_df['FareBand'] = pd.qcut(train_df['Fare'], 4)
train_df[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

FareBandに基づいて、Fare特徴量を序数値に変換します。


In [ ]:
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

train_df = train_df.drop(['FareBand'], axis=1)
combine = [train_df, test_df]
    
train_df.head(10)

## 5. モデル選択
これで予測する準備が整いました。予測モデルはたくさんあるので、問題のタイプと解決したい要件を理解したうえで評価できるモデルを選択する必要があります。今回の問題は分類と回帰の問題で、出力（生存しているかどうか）と他の変数や特徴量（性別、年齢、港...）との間の関係を考慮する必要があります。また、与えられたデータでモデルを学習させる教師あり学習と呼ばれる機械学習アルゴリズムを使用します。これらの基準-教師あり学習かつ分類と回帰とわかればモデルの選択を絞ることができます。このようなケースに当てはまるモデルは次のようなものがあります。

・ロジスティック回帰

・KNNまたはk-Nearest Neighbors

・サポートベクターマシン(SVM)

・決定木

・ランダムフォレスト

・パーセプトロン

・ニューラルネットワーク

・RVM

それぞれのモデルで予測モデルを構築してみましょう。

In [ ]:
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

### 5-1. ロジスティック回帰
ロジスティック回帰は、累積ロジスティック分布であるロジスティック関数を用いて確率を推定します。


In [ ]:
# Logistic Regression

logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

ロジスティック回帰を使用して、仮説を検証することができます。これは、係数を計算することで確認できます。

正の係数は対数オッズを増加させ(確率を増加させ)、負の係数は対数オッズを減少させ(確率を減少させ)ます。

・Sexが最も高い正の係数で、Sexの値が増加すると（男性：0から女性：1）、Survived=1の確率が最も高くなることがわかる。

・逆にPclassが高くなると、生存している確率は最も低くなる。

・このように、Age×PClassはSurvivedと2番目に高い負の相関があるので、モデル化するのに適した特徴量であることがわかる。

・また、Nameから作成した特徴量=タイトルも2番目に高い正の相関がある。


In [ ]:
coeff_df = pd.DataFrame(train_df.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

### 5-2. SVM 
次にサポートベクターマシンを用いてモデル化します。

このモデルは、ロジスティクス回帰モデルよりも高いスコアを生成できました。


In [ ]:
# Support Vector Machines

svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

### 5-3. k-NN 
パターン認識では、k-近傍法アルゴリズム（略してk-NN）は、分類と回帰に使用されるノンパラメトリック手法です。

KNNの信頼度スコアは、ロジスティック回帰よりは良いが、SVMよりは悪い。


In [ ]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

### 5-4. パーセプトロン
パーセプトロンとは、2値分類器（数値のベクトルで表される入力が、ある特定のクラスに属するか否かを判断できる関数）の教師付き学習のためのアルゴリズムである。これは線形分類器の一種であり，特徴ベクトルと重みを組み合わせた線形予測関数に基づいて予測を行う分類アルゴリズムです。


In [ ]:
# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron

In [ ]:
# Linear SVC

linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
acc_linear_svc

In [ ]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
acc_sgd

### 5-5. 決定木

モデルスコアは、これまでに評価されたモデルの中で最も高い。


In [ ]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

### 5-6. ランダムフォレスト
ランダムフォレストは、人気のあるモデルの一つです。ランダムフォレストまたはランダム決定木とは、分類や回帰などのためのアンサンブル学習法で、学習時に多数の決定木（n_estimators=100）を構築し、個々の木のクラス（分類）や平均予測（回帰）のモードとなるクラスを出力することで動作します。

モデルのスコアは、これまでに評価されたモデルの中で最も高い。
このモデルの出力(Y_pred)を提出用として利用することにした。


In [ ]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

## 6. モデルの評価
すべてのモデルの評価を比較して最も適したモデルを選択します。決定木とランダムフォレストのスコアは同じですが、決定木の過学習する傾向があるため、ランダムフォレストを使用することにした。


In [ ]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree]})
sorted_model=models.sort_values(by='Score', ascending=False)
sorted_model

In [ ]:
plt.figure(figsize=(20,10))
fig = plt.bar(sorted_model['Model'], sorted_model['Score'],color='aqua')
plt.grid()
plt.show()

In [ ]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })
submission.to_csv('submission2.csv', index=False)

## References

This notebook has been created based on great work done solving the Titanic competition and other sources.

- [A journey through Titanic](https://www.kaggle.com/omarelgabry/titanic/a-journey-through-titanic)
- [Getting Started with Pandas: Kaggle's Titanic Competition](https://www.kaggle.com/c/titanic/details/getting-started-with-random-forests)
- [Titanic Best Working Classifier](https://www.kaggle.com/sinakhorami/titanic/titanic-best-working-classifier)

![](https://i.imgur.com/2BUDtOe.jpg)